## Hidden Markov Model

- Gaussian Mixture Model에 시간적인 특성(비가역적인)을 넣어보자

- 이를 GMM의 Bayesan Notation에서 HMM과 비교해보자.

<img  src="./image/W9-1.PNG" width="100%">

- Multinomial로 modelling되는 파이라는 파라미터가 결정되면 z1과 z3가 독립이되어 시간의 특성을 표현하는데 잘 맞지 않겠다 하여 오른 쪽과 이 모델링 해준다.

- 따라서 Temporal Causality를 부여해줄 수 있으며, HMM을 Dynamic Clustering이라고 한다.

- HMM 은 Discrete한 데이터나 Continuous한 데이터 모두 다룰 수 있는 모델이지만, discrete만 다룰 것이며, 그리고 latent factor 또한 Continuous할 수 있는데 이를 ** Kalman filter ** 라고 하는데, 이는 다루지 않을 것이다.

- 시간에 따라서 lataent와 observation이 구분되는 것이 특징

### Probabilities in HMM

- (in discrete environment)

<img  src="./image/W9-2.PNG" width="40%">

- Initial State Probabilities:
    - z1은 multinomial distribution에서 sampling되며, multinomial은 pi라는 파라미터로 가져 이를 Learning하거나 정해져 있어야한다.
    - $ P(z_{1}) \sim Mult(\pi_{1}), ... , \pi_{k}) $
    

- Transition Probabilities
    - 다른 latent(클러스터)로 갈 확률
    - $ P(z_{t} | z_{t-1}^{i} = 1 ) \sim Mult(a_{i,1}), ... , a_{i,k})  $
    - or $ P(z_{t}^{j} = 1 | z_{t-1}^{i} = 1 ) = a_{i,j} $
    - i번째 latent에 있는데 j번째 latent로 갈 확률


- Emission Probabilities
    - i번 째 latent에 있는데, 보여지는 observation x는 어떤 것인지
    - $ P(x_{t} | z_{t}^{i} = 1 ) \sim Mult(b_{i,1}), ... , b_{i,m}) \sim f(x_{t}|\theta_{i}) $
        - m개의 possible한 observation
    - or, $ P(x_{t}^{j} = 1 | z_{t}^{i} = 1 )  = b_{i,j}  $
    - i번째 latent에 있을 때 j번째 observation이 나올 확률
    

### Main Questions on HMM

- $ \pi $는 initial state prob. $ a $ 는 Transition Prob , $ b $는 Emission prob. $ X $는 observation, $ M $ 은 topology of HMM에 대해
 
 
- A. Evaluation Question: 관측된 데이터가 얼마나 likely 한지
    - How much is X likely to be observed in the trained model?
    - Given: $ \pi, a, b, X $
    - Find: $ P(X|M,\pi, a, b) $


- B. Decoding Question: 관측된 데이터의 흐름의 가장 likely한 latent factor의 sequence는 무엇인지.
    - What would be the most probable sequence of latent states?
    - Given: $ \pi, a, b, X $
    - Find: $ argmax_{z} P(Z|X,M,\pi, a, b) $

  
- C. Learning Question: 관측 데이터만 주어졌을 때, 주어진 데이터가 보여질
    - What would be the underlying parameters of the HMM given the observations?
    - Given: $ X $
    - Find: $ argmax_{\pi, a, b} P(X|M,\pi, a, b) $
    - GMM에서도 X만 주어졌었다. Z를 marginalize out하며 이 과정에서 EM을 사용할 것이다.
    
    
- Evaluation Q.는 counting,  Decoding Q.은 $ \pi, a, b $를 알아야하며, Learning Q.은 $ \pi, a, b  $ 를 알아보는 것이다.
   
   
### A) Answering Evaluation Question
  
- 카지노에서 딜러가 무게 중심이 삐딱한 Loaded Dice와 일반적인 Fair Dice를 섞어서 사용했을 때

- Obtaining  $ \pi, a, b $ from given X and M은
    - $ P(z_{t+1} = L | z_{t} = L )  $ (이전에 L-Dice를 쓰고 L-Dice를 사용할 확률) 등을 count해서 [W1 MLE & MAP 계산](https://github.com/NamSahng/Summary/blob/master/Intro2AI%26ML/W1%20MLE%20%26%20MAP.ipynb) 을 활용해 $ a $를 구할 수 있고
    - 이와 비슷하게 counting을 활용하고 MLE MAP를 통해 확률을 부여해 $ \pi, b $를 구할 수 있다.

<img  src="./image/W9-2.PNG" width="40%">

- 그리고 이를 위와 같은 topology라고 가정하고 Evaluation Q.에 답해보자.
- 위와 같은 topology의 joint probability와 그에 대한 Factorization은 다음과 같으며 이는 initial, transition, emission 확률들로 이루어져있음을 알 수 있다.
    - $ P(X,Z) = P(z_{1})P(x_{1}|z_{1})P(z_{2}|z_{1})P(x_{2}|z_{2})P(z_{3}|z_{2})P(x_{3}|z_{3}) $
    - $ = \pi_{idx(z_{1}=1)}b_{idx(x_{1}=1),idx(z_{1}=1)}a_{idx(z_{1}=1),idx(z_{2}=1)}  \cdots$

- 그리고 166의 seq로 observation이 나왔다 하고, 이에대한 Latent(Dice)가 LLL 또는 FFF를 evaluation해 보면 (loaded는 6이 나올 확률이 1/2 나머지는 1/10이며, a에서 L to L은  0.7 F to F는 0.5 로 MLE MAP를 통해 확률 을 구했다고 가정)

<img  src="./image/W9-3.PNG" width="50%">

- 그런데 이렇게 모든 Latent(어떤 dice였는지)에 대한 seq를 알려면, 계산량이 지수적으로 증가하기 때문에 알기 어렵다. 따라서, Decoding Question에서(그중 Forward Probability Calculation) 알아내는 방법을 배우자.


### B) Answering Decoding Question


<img  src="./image/W9-4.PNG" width="100%">

- Marginal Probability를 Recursive한 form으로 표현해보자.
    - $ P(A,B,C) = P(A)P(A|B)P(C|A,B) $를 이용
    
<img  src="./image/W9-5.PNG" width="40%">

- 파란색 부분만 확률을 표현하고, 빨간색 부분을 marginalization을 해서 재귀적 표현으로 만들어보면,
    - $ z_{t-1} $이 given이면 $ z_{t} $ 를 결정하는데 있어 $ x_{1} $부터 $ x_{t-1} $까지 알 필요가 없다. $ z_{t-1} $만 알면 independent하게 갈 수 있음(Conditiaonal Independece of the structure)을 이용.(2번째 줄 => 3번째 줄)
        
<img  src="./image/W9-6.PNG" width="80%">

- 그리고 이 Recursive한 것을 Recursion으로 풀면 넘 힘드니, 이를 풀 때 유용한 DP의 Memoization을 활용하자. (Top Down => Bottom Up approach로)


- ** Forward Probability Calculation **
    - 위에서 정의한 $ \alpha_{t}^{k} $ 가 Forward Probability이며 이는 Time X States의 dimension을 갖고 있다.
    - Z를 모르지만, $ \alpha_{t}^{k} $와 X를 통해 Evaluation Question을 푸는 방법.
    - Initial:
        - $ \alpha_{t}^{k} = b_{k,x_{1}} \pi_{k} $
    - Iterate until time T: 시간을 점점 늘려감.
        - $ \alpha_{t}^{k} = b_{k,x_{t}} \sum_{i} \alpha_{t-1}^{i} a_{i,k} $
    - 그리고 이를 더한 $ \sum_{i} \alpha_{T}^{i} $를 return하면, Z가 없는 Evaluation Question을 답하는 것이다.
        - pf for correctness) $ \sum_{i} \alpha_{T}^{i} = P(x_{1},...,x_{T},z_{T}^{i}=1) = P(x_{1}, ..., x_{t}) $
        - Memoization Table은 $ \alpha_{t}^{k} $들을 계산하는데 사용.

- Limitation of the Forward Probability
    - 우리가 정한 t가 정말 T일 때는 이렇게 joint를 하여 $ P(x_{1}, ..., x_{t},z_{t}) $를 계산을 잘해서 $ z_{t} $를 assgin을 해 확률을 잘 구했다.
    - 그러나, BUT
    - $ t \neq T $ 일 때는 $ x_{t+1}, ...x_{T} $가 $ z_{t} $를 assign 할 때 고려해야한다. 이렇게 전체 X를 생각을해 특정 time에서의 latent factor를 알아볼 수 있는 Backward Probabaility를 정의해야하는구나.
        - Recall [Bayes Ball](https://github.com/NamSahng/Summary/blob/master/Intro2AI%26ML/W7%20Bayesian%20Network.ipynb)
        
       

- ** Backward Probability Calculation **
    - Forward Prob를 계산하는 것과 대동소이 하다.
    - <img  src="./image/W9-7.PNG" width="80%">
        - 1에서 2 번째 줄 joint prob의 정의: P(A,B) = P(A)P(B|A)
        - 2에서 3번째는 topology에서 $ z_{t}^{k} $ 가 given이면 조건부독립 성립
        - 앞의 forward prob의 형태가 나오고, $ \beta_{t}^{k} $라는 term으로 하고, 이를 Backward Probability라 할 것이다.
    - $ \beta_{t}^{k} = P(x_{t+1} , ... , x_{T}| z_{t}^{k} = 1 ) $ 에서 $ z_{t+1} $을 추가(introduce)하고, marginalization해서 recursive한 형태로 만들어 보자.
    - <img  src="./image/W9-8.PNG" width="100%">
        - $  \beta_{t+1}^{k} $은 더 작다.(t+2부터 시작하니까)
        - 그리고 이도 Recursive하니까 DP로 풀 수 있다.
    - 그래서 결국, 정리를 하면 Whole Seq observation을 고려한 한 time의 latent는 forward와 backward의 곱으로 표현된다.
    - $ P(z_{t}^{k} = 1 , X) = \alpha_{t}^{k}\beta_{t}^{k} = (b_{k,x_{t}} \sum_{i} \alpha_{t-1}^{i} a_{i,k}) \space  \times \space (\sum_{i}a_{k,i}b_{i,x_{t}} \beta_{t+1}^{i} )   $
    - 이를 활용해서 Decoding 문제를 풀어보자.


- $ P(z_{t}^{k} = 1 , X) = \alpha_{t}^{k}\beta_{t}^{k} = (b_{k,x_{t}} \sum_{i} \alpha_{t-1}^{i} a_{i,k}) \space  \times \space (\sum_{i}a_{k,i}b_{i,x_{t}} \beta_{t+1}^{i} ) $
- forward backwad prop를 통해 여기 까지 하면, 한 시점에서의 latent만 알 수 있다.
- Z에 대한 whole sequence를 알아보자. $ \to $ Decoding Question.


- ** Viterbi Decoding **
    - $ k^{*} = argmax_{k} P(z^{k} = 1 | X) = argmax_{k} P(z^{k} = 1 , X) $ 
        - k-star는 most proabable assignment of 특정 클러스터 k (전체 시간에 대한)
        - conditional은 noramalizing term을 빼면 joint를 maximize하는 것은 같다.
    - z또한 처음 시점 t부터 DP를 활용해 memoization table을 만들며 채워나가자.
        - using Forward Approach (bottom-up)
    - $ V_{t}^{k} = max_{z_{1}\cdots z_{t-1}}P(x_{1}, \cdots , x_{t-1}, z_{1}, \cdots , z_{t-1}, x_{t}, z_{t}^{k} = 1) $
        - $  V_{t}^{k} $: Most probable sequence of latent states until t-1 and fixing the state k at time t
    - $ = max_{z_{1}\cdots z_{t-1}}P( x_{t}, z_{t}^{k} = 1 | x_{1}, \cdots , x_{t-1}, z_{1}, \cdots , z_{t-1})P(x_{1}, \cdots , x_{t-1}, z_{1}, \cdots , z_{t-1}) $
         - Factorize
    - $ = max_{z_{1}\cdots z_{t-1}}P( x_{t}, z_{t}^{k} = 1 | z_{t-1})P(x_{1}, \cdots , x_{t-2}, z_{1}, \cdots , z_{t-2} , x_{t-1}, z_{t-1}) $
        - Conditional Independence, 뒤 쪽은 repeating되게 살짝 뒤로 빼줌. 
    - $ = max_{z_{t-1}}P( x_{t}, z_{t}^{k} = 1 | z_{t-1}) max_{z_{1}\cdots z_{t-2}} P(x_{1}, \cdots , x_{t-2}, z_{1}, \cdots , z_{t-2} , x_{t-1}, z_{t-1}) $
        - maximization loop중 t-1 빼냄
    - $ = max_{i \in z_{t-1}}P( x_{t}, z_{t}^{k} = 1 | z_{t-1}^{i} = 1)V_{t-1}^{i}  $
        - 이전 time의 assignemt의 i로 표현. 뒷부분은 V로 표현
    - $ = max_{i \in z_{t-1}}P( x_{t} | z_{t}^{k} = 1) P(z_{t}^{k} = 1|z_{t-1}^{i} = 1) V_{t-1}^{i}  $
        - emmission과 transition으로 바꿔줌. 그리고 아래는 상관없는 emission part를 앞으로 빼주고, a, b로 표현하면,
    - $ = P( x_{t} | z_{t}^{k} = 1) max_{i \in z_{t-1}} P(z_{t}^{k} = 1|z_{t-1}^{i} = 1) V_{t-1}^{i}    $
    - $ = b_{k,idx(x_{t})} max_{i \in z_{t-1}} a_{i,k} V_{t-1}^{i}  $
        -  $ V_{t}^{k} $이 recursive하니까 DP를 활용해 T까지 계산하면, 전체 sequence에 대한 most probable assign을 얻을 수 있다. $ V_{t}^{k} $
    - 맨 아래와 같이 trace하고 retrace한다. 아래는 최소 거리였다면, 여기에 확률을 넣어 최대 확률로하자.
    - <img  src="./image/W9-9.PNG" width="90%">
    - 위의 DP를 활용한 trace와 retrace에 대해  $ V_{t}^{k} $에 대해 적용하면, Trace와 확률을 2가지를 저장.
    - ** Viterbi Decoding Algorithm **
        - Initialize : $ V_{1}^{k} = b_{k,x_{1}}\pi_{k} $
        - Iterate until time T: (a, b는 이미 forward, backward prob calc로 계산해놓았다.)
            - $ V_{t}^{k} = b_{k,idx(x_{t})}max_{i\in z_{t-1}}a_{i,k} V_{t-1}^{i} \quad \to $ 계산하고 저장 (위 그림의 왼쪽)
            - $ trace_{t}^{k} = argmax_{i\in z_{t-1}}a_{i,k} V_{t-1}^{i}   \quad \to $ trace를 이를 저장. (위 그림의 오른쪽)
        - Return $ P(X,Z^{*}) = max_{k}V_{T}^{k},z_{T}^{*} = argmax_{k}V_{T}^{K},z_{t-1}^{*} = trace_{t}^{z^{*}} \quad \to  $ 이렇게 $ V_{T}^{k} $가 최대가되는 k를 찾아 retrace를 통해 whole seq를 찾아내는 과정으로 간다.
    - 이론적으로는 이렇게 끊나지만, Technical Difficulties로 underflow prob이 많이 발생하여 log domaing으로 하여 덧셈으로 계산한다.
    - 이제 given $ \pi, a, b, X $로 Decoding을 할 수 있으며, Pos tagging할 때 이를 활용할 수 있다.



### C) Answering Learning Question

- Evaluation(by forward algoritm) 과 Decoding(by viterbi algoritm)은 $ \pi, a, b $를 알고 있다는 것에 의존하며, X와 Z를 observed(supervision)를 알려주었어서 가능했다.

- Z를 모른다면, X를 통해 Z와 $ \pi, a, b $를 알아내자.
- Strategy:
    - Find optimized $ \hat{\pi}, \hat{a}, \hat{b} $ with $ X $
    - Find Most probable Z with $ X, \hat{\pi}, \hat{a}, \hat{b}  $ (viterbi decoding!)
    - $ \to $ EM을 활용해 iteratively하게 optimze하자!
    
    
- ** Baum Welch Algorithm ** (EM for HMM, Forward-Bacward라고 부르기도함)
    - Initialize $ \pi^{0}, a^{0}, b^{0} $ to an arbitrary poitnt(너무 막잡으면 local optima 위험)
    - Loop EM until likelihood converges
    - <img  src="./image/W9-10.PNG" width="80%">
    
- 그리고 이를 수식으로 나타내면, (파이에 대해서만 유도하심, 11분)[유도 영상](https://www.edwith.org/machinelearning2__17/lecture/10872/)

- Expectation은 위에서 배운 것들을 이용해 Viterbi decoding까지 배운 개념을 적용하고
- Maximization에서는 $ \pi, a, b $에대해 편미분을 통해 구하여 구하면 다음과 같다.
    - 왼쪽의 t들은 모두 iterate할 때 t이며 오른쪽은 sequence의 t이다.
<img  src="./image/W9-11.PNG" width="90%">


- 다음 시간에는 MCMC계열의 sampling based의 inference를 배우자. 



<br><br>
### Reference:

- 문일철 교수님, 인공지능 및 기계학습 개론 II, https://www.edwith.org/machinelearning2__17

- 참고: Stephen Tu, Derivation of Baum-Welch Algorithm for Hidden Markov Models
, https://people.eecs.berkeley.edu/~stephentu/writeups/hmm-baum-welch-derivation.pdf

<br><br>
### Quiz
- Q1 <img  src="./image/W9-q1.PNG" width="90%">

- Q2 <img  src="./image/W9-q2.PNG" width="90%">

- Q3 <img  src="./image/W9-q3.PNG" width="90%">